In [1]:
import openml, fairlib
import fairlib as fl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer
from keras.models import Sequential
from keras.layers import Dense
from fairlib.inprocessing import Fauci

2024-11-12 17:48:34.751411: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731430114.770638 4158716 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731430114.776398 4158716 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 17:48:34.796846: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:fairlib:Using Keras backend: TENSORFLOW
INFO:fairlib:fairlib loaded


In [2]:
dataset = openml.datasets.get_dataset(179)
X, y, _, names = dataset.get_data(target=dataset.default_target_attribute)

INFO:openml.datasets.dataset:pickle write adult


In [3]:
imputer = SimpleImputer(strategy='most_frequent')
X_imputed = imputer.fit_transform(X)

In [4]:
X_discretized = X_imputed.copy()
for col in X.columns:
    if X[col].dtype == 'category':
        le = LabelEncoder()
        X_discretized[:, X.columns.get_loc(col)] = le.fit_transform(X_discretized[:, X.columns.get_loc(col)])


In [5]:
X = fairlib.DataFrame(X_discretized, columns=names)
y = y.apply(lambda x: x == ">50K").astype(int)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
X_train['income'] = y_train

In [8]:
X_train

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native-country,income
5551,3,1,277024.0,11,9,5,10,1,2,1,0,0,2,38,0
36721,4,4,164616.0,10,16,2,9,0,4,1,4,0,2,38,1
2638,3,3,159755.0,8,11,2,9,0,4,1,0,0,3,38,1
36214,3,3,260052.0,9,13,0,3,4,4,1,4,0,2,38,1
27010,2,4,188615.0,9,13,2,11,0,4,1,0,0,3,38,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,1,3,200117.0,9,13,2,3,0,1,1,0,2,3,38,1
44732,0,3,90896.0,11,9,4,6,3,4,0,0,0,2,38,0
38158,0,3,370057.0,11,9,0,0,1,4,0,0,0,2,38,0
860,0,3,216284.0,1,7,4,0,3,4,0,0,0,0,38,0


In [9]:
fauci_train_dataset = fl.DataFrame(X_train)

In [10]:
fauci_train_dataset.targets = "income"
fauci_train_dataset.sensitive = 'sex' # fauci currently supports only one sensitive attribute

In [11]:
def create_model():
    model = Sequential()
    model.add(Dense(12, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [12]:
unprocessed = Fauci(create_model(), loss='binary_crossentropy', regularizer=None, optimizer='adam', metrics=['accuracy'], regularization_weight=0.0)
inprocessing_spd = Fauci(create_model(), loss='binary_crossentropy', regularizer='sp', optimizer='adam', metrics=['accuracy'])
inprocessing_di = Fauci(create_model(), loss='binary_crossentropy', regularizer='di', optimizer='adam', metrics=['accuracy'])

2024-11-12 17:48:36.858964: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [13]:
unprocessed.fit(fauci_train_dataset, converting_to_type=float, epochs=10, batch_size=32, validation_split=0.3)

Epoch 1/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.3961 - loss: 13788.7217 - val_accuracy: 0.7601 - val_loss: 0.8996
Epoch 2/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6616 - loss: 3.5753 - val_accuracy: 0.2399 - val_loss: 11.7419
Epoch 3/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6505 - loss: 4.3013 - val_accuracy: 0.7902 - val_loss: 0.5949
Epoch 4/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6460 - loss: 4.3228 - val_accuracy: 0.7601 - val_loss: 15.8051
Epoch 5/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6407 - loss: 5.7436 - val_accuracy: 0.7601 - val_loss: 3.7742
Epoch 6/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6462 - loss: 5.3111 - val_accuracy: 0.7625 - val_loss: 1.3580
Epoch 7/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6473 - loss: 3.8668 - val_accuracy: 0.7629 - val_loss: 1.0587
Epoch 8/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6489 - loss: 5.7940 - val_accura

In [14]:
inprocessing_spd.fit(fauci_train_dataset, converting_to_type=float, epochs=10, batch_size=32, validation_split=0.3)

Epoch 1/10
RICHIAMO LA SPD
RICHIAMO LA SPD
705/716 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6211 - loss: 1157.0601RICHIAMO LA SPD
716/716 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6228 - loss: 1142.1680 - val_accuracy: 0.7601 - val_loss: 0.3022
Epoch 2/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7613 - loss: 0.2951 - val_accuracy: 0.7601 - val_loss: 0.2820
Epoch 3/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7644 - loss: 0.2782 - val_accuracy: 0.7601 - val_loss: 0.2765
Epoch 4/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7684 - loss: 0.2718 - val_accuracy: 0.7601 - val_loss: 0.2755
Epoch 5/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7660 - loss: 0.2722 - val_accuracy: 0.7601 - val_loss: 0.2755
Epoch 6/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7610 - loss: 0.2750 - val_accuracy: 0.7601 - val_loss: 0.2755
Epoch 7/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7685 - loss: 0.2706 - val_accuracy: 0.

In [15]:

inprocessing_di.fit(fauci_train_dataset, converting_to_type=float, epochs=10, batch_size=32, validation_split=0.3)

Epoch 1/10
RICHIAMO LA DI
RICHIAMO LA DI
716/716 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6414 - loss: 48.9984RICHIAMO LA DI
716/716 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6414 - loss: 48.9963 - val_accuracy: 0.7601 - val_loss: 17.7360
Epoch 2/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6555 - loss: 48.1418 - val_accuracy: 0.2399 - val_loss: 13.0411
Epoch 3/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6418 - loss: 37.2495 - val_accuracy: 0.7433 - val_loss: 2.8860
Epoch 4/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6397 - loss: 42.8239 - val_accuracy: 0.7585 - val_loss: 6.1209
Epoch 5/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6459 - loss: 42.5869 - val_accuracy: 0.7601 - val_loss: 81.4393
Epoch 6/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6423 - loss: 42.5367 - val_accuracy: 0.2399 - val_loss: 38.4144
Epoch 7/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6371 - loss: 34.7585 - val_accuracy:

In [16]:
y_pred_unprocessed = unprocessed.predict(X_test.astype(float))
y_pred_spd = inprocessing_spd.predict(X_test.astype(float))
y_pred_di = inprocessing_di.predict(X_test.astype(float))

504/504 ━━━━━━━━━━━━━━━━━━━━ 0s 731us/step
504/504 ━━━━━━━━━━━━━━━━━━━━ 0s 705us/step
504/504 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step


In [17]:
print(y_pred_unprocessed.sum())
print(y_pred_spd.sum())
print(y_pred_di.sum())

4793.1133
3808.7876
5.8812175


In [18]:
y_pred_unprocessed = (y_pred_unprocessed > 0.5).astype(int)
y_pred_spd = (y_pred_spd > 0.5).astype(int)
y_pred_di = (y_pred_di > 0.5).astype(int)

In [19]:
print(y_pred_unprocessed.mean())
print(y_pred_spd.mean())
print(y_pred_di.mean())

0.08983744881498945
0.0
0.0003722546221615585


In [20]:
from sklearn.metrics import accuracy_score
print("accuracy : ", accuracy_score(y_test, y_pred_unprocessed))
print("accuracy with spd regularization : ", accuracy_score(y_test, y_pred_spd))
print("accuracy with di regularization : ", accuracy_score(y_test, y_pred_di))

accuracy :  0.7788807544360342
accuracy with spd regularization :  0.75716590147661
accuracy with di regularization :  0.7575381560987715


In [21]:
X_test['income'] = y_pred_spd

In [22]:
spd_dataset = fl.DataFrame(X_test)
spd_dataset.targets = "income"
spd_dataset.sensitive = 'sex'

spd_dataset.statistical_parity_difference()

(income=0, sex=1) -> 0.0
(income=0, sex=0) -> 0.0

In [23]:
X_test['income'] = y_pred_di

In [24]:
di_dataset = fl.DataFrame(X_test)
di_dataset.targets = "income"
di_dataset.sensitive = 'sex'

di_dataset.disparate_impact()

this is unprivileged in di:  0.0
this is privileged in di:  0.0005568445475638051
this is unprivileged in di:  0.0005568445475638051
this is privileged in di:  0.0


/home/gciatto/Work/Studenti/master-thesis-dizio-ay2324/fairlib/metrics/__init__.py:114: RuntimeWarning: divide by zero encountered in scalar divide
  di = unprivileged_rate / privileged_rate


(income=1, sex=1) -> 0.0
(income=1, sex=0) -> inf

## Test With un-fair model

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [25]:
model = create_model()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
X_train = X_train.astype(float)
y_train = y_train.astype(float)

In [30]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.3)

Epoch 1/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6682 - loss: 15.6222 - val_accuracy: 0.7601 - val_loss: 53.1400
Epoch 2/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6648 - loss: 26.9077 - val_accuracy: 0.2402 - val_loss: 56.4027
Epoch 3/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6636 - loss: 28.2206 - val_accuracy: 0.5021 - val_loss: 2.4930
Epoch 4/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6835 - loss: 11.8082 - val_accuracy: 0.7603 - val_loss: 41.9019
Epoch 5/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6673 - loss: 25.0679 - val_accuracy: 0.8487 - val_loss: 0.5395
Epoch 6/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6899 - loss: 14.0622 - val_accuracy: 0.7616 - val_loss: 27.1700
Epoch 7/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6969 - loss: 12.2731 - val_accuracy: 0.7691 - val_loss: 12.6557
Epoch 8/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6932 - loss: 14.6926 - val

In [31]:
y_pred = model.predict(X_test.astype(float))
y_pred = (y_pred > 0.5).astype(int)

504/504 ━━━━━━━━━━━━━━━━━━━━ 0s 684us/step


In [32]:
accuracy_score(y_test, y_pred)

0.7626256359349796

In [33]:
X_test['income'] = y_pred
spd_dataset = fl.DataFrame(X_test)
spd_dataset.targets = "income"
spd_dataset.sensitive = 'sex'

spd_dataset.statistical_parity_difference()

(income=1, sex=1) -> 0.012046414134242473
(income=1, sex=0) -> -0.012046414134242473

In [34]:
spd_dataset.disparate_impact()

this is unprivileged in di:  0.0022459292532285235
this is privileged in di:  0.014292343387470997
this is unprivileged in di:  0.014292343387470997
this is privileged in di:  0.0022459292532285235


(income=1, sex=1) -> 0.15714212794504767
(income=1, sex=0) -> 6.363665893271461